# Word of the Day

This notebook fetches the word every day from the site http://www.dictionary.com/wordoftheday/ 

What it results into:

1. Name of the word
2. Category of the word
3. Meanings of the word
4. Origin of the Word
5. Usage Examples from the Web



In [10]:
# Import the necessary libraries

import requests
from bs4 import BeautifulSoup
import os
import datetime
import warnings

warnings.filterwarnings('ignore')
url = "http://www.dictionary.com/wordoftheday/"


# Get the date of previous day.

date = datetime.datetime.now() - datetime.timedelta(days=1)
date_format = str(date.year) + '/' + str(date.month) + '/' + str(date.day)

newurl = url + date_format

response = requests.get(newurl)
status_code = response.status_code 

if(~status_code == 200):
    print("The page you are trying to view is giving the error {}".format(status_code))
    quit()

# Using BeautifulSoup
soup = BeautifulSoup(response.content)

word = soup.find('div', class_ = 'tile-image')
word.a['href']

#Get the URL of each word. Each word has a separate page on the site.

word_url = word.a['href']


# Make a request to the word URL

response = requests.get(word_url)
soup = BeautifulSoup(response.content)

status_code = response.status_code 
if(~status_code == 200):
    print("The page you are trying to view is giving the error {}".format(status_code))
    quit()

In [11]:
# Get the name of the word

soup = BeautifulSoup(response.content)

word = soup.find('h1', class_= 'head-entry')
name_of_word = word.contents[0].contents[0]
name_of_word

'muckrake'

In [12]:
# Get the category of the word

def_list = soup.find('div', class_ = 'def-list')
print(type(def_list))
category = def_list.contents[1].span.contents[0]
category

<class 'bs4.element.Tag'>


'verb (used without object)'

In [13]:
# Get the indices of the meanings of the word

numbers = def_list.contents[1].find_all('span', class_ = 'def-number')
print(type(numbers))
numbers_list = [x.contents[0] for x in numbers]
numbers_list

<class 'bs4.element.ResultSet'>


['1.']

In [14]:
# Get the meanings of the words

meanings = def_list.contents[1].find_all('div', class_ = 'def-content')
meaning_list = [x.contents[0].strip() for x in meanings]
meaning_list

['to search for and expose real or alleged corruption, scandal, or the like, especially in politics.']

In [15]:
# Append the indices and meanings of the word
new_meanings_list = []
for a,b in zip(numbers_list, meaning_list):
    new_meanings_list.append('{}{}'.format(a,b))
new_meanings_list

['1.to search for and expose real or alleged corruption, scandal, or the like, especially in politics.']

In [16]:
# Get the origin of the words

origin_languages = soup.find_all('div', class_ = 'map-origin-language')
origin = []
for o in origin_languages:
    origin.append(o.find('a').contents[0])
origin

[]

In [23]:
# Get the Examples from the Web
import re
examples = soup.find_all('p', class_= 'partner-example-text')

web_usage = []
for text in examples:
    web = [] 
    for t in text:
        web.append(str(t))
    final_word = "".join(web).strip()
    final_word = re.sub(r'<em>', '', final_word)
    final_word = re.sub(r'</em>', '', final_word)
   
    web_usage.append(final_word)
web_usage

['Of Walt Whitman it is: "The dirty, clumsy paws of a harper whose plectrum is a muckrake."',
 'The journalist might well exchange the muckrake for the pick and dig it out.']

# Here are the results

In [26]:
f = open('results.txt', 'w+')
f.write("\n\nDate: {} \n".format(date.strftime('%Y-%m-%d')))
f.write("\nWord: {} \n".format(name_of_word))
f.write("\nCategory: {} \n".format(category))
f.write("\nMeanings: \n\n")
for x in new_meanings_list:
    f.write(x)
    f.write('\n')
i = 1
f.write("\nExample Usages from the Web: \n \n")
for x in web_usage:
    f.write('{}.{}'.format(i,x))
    f.write('\n')
    i+=1

In [27]:
import pdfkit

pdfkit.from_file('results.txt', 'newresult.pdf')

Loading pages (1/6)
Counting pages (2/6)                                               
Resolving links (4/6)                                                       
Loading headers and footers (5/6)                                           
Printing pages (6/6)
Done                                                                      


True

In [ ]:
# Import smtplib for the actual sending function
import smtplib

# Import the email modules we'll need
from email.mime.text import MIMEText



# Open a plain text file for reading.  For this example, assume that
# the text file contains only ASCII characters.
with open('results.txt') as fp:
    # Create a text/plain message
    msg = MIMEText(fp.read())

me = "nitinvijay23@gmail.com"
you = 'nitinvijayy@gmail.com'
msg['Subject'] = 'The contents of '
msg['From'] = me
msg['To'] = you

# Send the message via our own SMTP server.
s = smtplib.SMTP('127.0.0.1')
s.send_message(msg)
s.quit()

In [7]:
import smtplib
server = smtplib.SMTP('smtp.gmail.com', 587)
server.ehlo()
server.starttls()

server.login("me@gmail.com", "pwd")
 
msg = "YOUR MESSAGE!"
server.sendmail("me@gmail.com", "you@gmail.com", msg)
server.quit()

SMTPAuthenticationError: (534, b'5.7.9 Please log in with your web browser and then try again. Learn more at\n5.7.9  https://support.google.com/mail/?p=WebLoginRequired b67sm146842pfj.81 - gsmtp')